# Tarea 4

**Envío del PR inicial:** viernes 28 de septiembre

**Aceptación del PR:** martes 9 de octubre

In [2]:
using Plots
using PlotRecipes
using LaTeXStrings
gr()

#"Módulo" Diferenciación Automática
import Base: +, -, *, /, ^
import Base.show
import Base: sqrt, exp, log, sin, cos, sinh, cosh

"""
    Dual

Definición de los números duales. Los campos internos son
    x  :: Float64   # valor de la función
    x′ :: Float64   # valor de su derivada

"""
struct Dual
    x  :: Float64
    x′ :: Float64
end

Dual(c::Real) = Dual(c, 0.0)
dual(x0::Real) = Dual(x0, 1.0)

+(u::Dual, v::Dual) = Dual( u.x + v.x, u.x′ + v.x′)

-(u::Dual, v::Dual) = Dual( u.x - v.x, u.x′ - v.x′)

*(u::Dual, v::Dual) = Dual( u.x * v.x, u.x * v.x′ + u.x′ * v.x)

function /(u::Dual, v::Dual)
    y = u.x / v.x
    Dual( y, (u.x′ - y * v.x′)/v.x )
end

function ^(a::Dual, n::Int)
    y = a.x^(n-1)
    Dual(a.x * y, n*y*a.x′)
end

#+
+(a::Real, u::Dual) = Dual( a + u.x, u.x′)
+(u::Dual, a::Real) = Dual( a + u.x, u.x′)
#-
-(a::Real, u::Dual) = Dual( a - u.x, -u.x′)
-(u::Dual, a::Real) = Dual( u.x - a, u.x′)
#*
*(a::Real, u::Dual) = Dual(u.x*a,u.x′*a)
*(u::Dual, a::Real) = Dual(u.x*a,u.x′*a)
#/
function /(a::Real,u::Dual)
    y = a / u.x
    Dual( y, (-y * u.x′)/u.x )
end
/(u::Dual, a::Real)=Dual( u.x/a, u.x′/a )

function show(u::Dual)
    print(string(u.x)*" + "*string(u.x′)*"ε")
end

sqrt(u::Dual)=Dual(sqrt(u.x),u.x′/(2*sqrt(u.x)))
exp(u::Dual)=Dual(exp(u.x),u.x′*exp(u.x))
log(u::Dual)=Dual(log(u.x),u.x′/u.x)
sin(u::Dual)=Dual(sin(u.x),u.x′*cos(u.x))
cos(u::Dual)=Dual(cos(u.x),-u.x′*sen(u.x))
sinh(u::Dual)=Dual(sinh(u.x),u.x′*cosh(u.x))
cosh(u::Dual)=Dual(cosh(u.x),u.x′*sinh(u.x))

"Método de Newton con Duales a partir de una función real f y un valor inicial x=0."
function newton(f,x0,tol=1e-15)
    fprime(x)=f(dual(x)).x′
    xn=x0
    while abs(f(xn))>tol
        xn=xn-f(xn)/fprime(xn)
    end
    return xn
end

"A partir de una función real f, y un rango x0, devuelven las raíces encontradas"
function soluciones(f,x0)
    raiz=[]
    raices=[]
    for i in x0
        r=newton(f,i)
        if abs(r)≠Inf
            push!(raiz,r)
        end
    end
    push!(raices,raiz[1])
    for i in 1:size(raiz,1)-1
        if abs(raiz[i]-raiz[i+1])>1e-10
            a=true
            b=true
            for j in raices
                if abs(raiz[i+1]-j)>1e-10
                    a=true
                else
                    a=false
                b=(a & b)
                end
            end
            if b==true
                push!(raices,raiz[i+1])
            end
        end
    end
    return raices
end


function mapeo(f,x0,iteraciones)
    x=x0
    for i in 1:iteraciones
        x=f(x)
    end
    return x
end

function conjunto_mapeos(f,x,n)
    y=[]
    for i in x
        push!(y,mapeo(f,i,n))
    end
    return y
end

function orbitas(f,x0,n)
    x0s=[]
    fs=[]
    x02s=[]
    f2s=[]
    for i in 1:n
        push!(x0s,x0)
        push!(x02s,x0)
        push!(x02s,x0)
        x0=f(x0)
        push!(f2s,x0)
        push!(f2s,x0)
        push!(fs,x0)
    end
    a=minimum(x0s)
    a=floor(a)
    b=maximum(x0s)
    b=floor(1+b)
    rangox=a:(1/(2^10)):b
    y=f.(rangox)
    x02s=x02s[1:2n]
    pop!(f2s)
    insert!(f2s,1,x02s[1])
    return rangox, x0s, fs, x02s, f2s
end

"""
Se traza un diagrama con los siguientes parámetros:
f: La función a trazar e iterar.
rango: El conjunto de puntos iniciales desde los que se itera.
pasos: El número de iteraciones a realzar.
recta: true para una recta positiva, false para una recta negativa.
puntos: true para marcar los puntos intermedios de la trayectoria, false para no marcarlos y señalar sólo el inicial y final.
"""
function diagramaorbitas(f,rango,pasos,recta=true,puntos=false)
    rangox, x0s, fs, x02s, f2s=orbitas(f,rango[1],pasos)
        if recta==true
        plot(rangox,rangox,color=:green)
        else
        plot(-rangox,rangox,color=:green)
        end    
        for j in rango
        rangox, x0s, fs, x02s, f2s=orbitas(f,j,pasos)
        a=rand()
        b=rand()
        c=rand()
        n=length(x0s)
        k=length(x02s)
        plot!(rangox,f,legend=false,color=:blue)
        if recta==true
            plot!(rangox,rangox,color=:green)
        else
            plot!(-rangox,rangox,color=:green)
        end
        if puntos==true
            for i in 1:(n-1)
                plot!([x0s[i]],[fs[i]],marker=(:dot),color=RGB(a,b,c))
            end
            plot!([x02s[1]],[x02s[1]],marker=(:dot,7),color=RGB(a,b,c))
            plot!([x02s[k]],[f2s[k]],marker=(:dot,7),color=RGB(a,b,c))
        else
            plot!([x02s[1]],[x02s[1]],marker=(:dot),color=RGB(a,b,c))
            plot!([x02s[k]],[f2s[k]],marker=(:dot),color=RGB(a,b,c))
        end
        plot!(x02s,f2s,line=(:path,:dot),color=RGB(a,b,c))
    end
    plot!(rangox,f,legend=false,color=:blue,xaxis="x_0",yaxis="f(x_0)",title="Órbitas")
    
end

diagramaorbitas

In [8]:
c=-1
Qc(x)=x^2+c
fc(x)=Qc(x)-x
soluciones(fc,-1:0.1:2)

2-element Array{Any,1}:
 -0.618034
  1.61803 

## 1

Para el mapeo $Q_c(x) = x^2 + c$, ¿cuál es el dominio $\cal{D}\subset \mathbb{R}$ de $Q_c(x)$ tal que la órbita generada por una condición inicial $x_0\in \cal{D}$ *no* escapa a infinito a ningún tiempo?

In [10]:
diagramaorbitas(Qc,-2:0.25:2,3)
plot!(xlims=(-5,10),ylims=(-5,10))

Llamemos $M=max\{|p_+|,|p_-|\}=p_+$ y $L=min\{|p_+|,|p_-|\}=p_+$ 

podemos notar que como $f(x)$ sólo tiene soluciones sí

$c>1/4 \Rightarrow M>0.5$ y $f'(M)=2M>1 \forall M$

por lo que, para cualquier punto $x_0>M$ si $n\rightarrow N \Rightarrow f^n(x_0)\rightarrow \infty$; que nos da una cota superior para los x_0 que no se van a infinito. 

Ahora bien, si nos fijamos en los $x_0<0$ tales que $|x_0|>M$, tendremos que para la primera iteración 

$f(x_0<-M)>M \Rightarrow f^2(x_0<-M)=f(x_0>M)$ que cae en el caso anterior; por lo que tenemos una cota inferior ahora.

Verificamos ahora que en el conjunto $[-M,M]$ no hay puntos que se vayan al infinito. 

Si $M-\delta<x_0<M$ como M es un punto repulsor 

$0<f(x_0)<M$. Si $-M>x_0>-M+\delta$ entonces regresamos en la primera iteración a 

$0<f(x_0)<M$ por lo que regresamos al caso anterior. 

Ahora, x va a avanzar hasta antes de llegar a L, pues L es otro repulsor. 

Por lo que aseguramos que si $x_0 \in [-M,M] \Rightarrow n\rightarrow N f^n(x_0)\in [-M,M]$

## 2

- Obtengan el diagrama de bifurcación que se muestra en la Fig. 2 del mapeo $Q_c(x)$ a partir de la iteración de una condición inicial.

- Caractericen la cuenca de atracción de $p_-(c)$ cuando el punto es estable.


In [22]:
Q(c,x)=x^2+c
function bifurcuenca(f,a,x0,n)
    xf

(-1.0:0.0009765625:1.0, Any[0.5, -0.25, -0.4375, -0.308594, -0.40477, -0.336161, -0.386996, -0.350234, -0.377336, -0.357618  …  -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025], Any[-0.25, -0.4375, -0.308594, -0.40477, -0.336161, -0.386996, -0.350234, -0.377336, -0.357618, -0.37211  …  -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025], Any[0.5, 0.5, -0.25, -0.25, -0.4375, -0.4375, -0.308594, -0.308594, -0.40477, -0.40477  …  -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025], Any[0.5, -0.25, -0.25, -0.4375, -0.4375, -0.308594, -0.308594, -0.40477, -0.40477, -0.336161  …  -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025, -0.366025])

In [30]:
y=[2,3]
x=[75,-90]
X=["Hola","Adiós"]
histogram(x,y)


LoadError: [91msyntax: keyword argument is not a symbol: ":string"[39m

## 3

Para el mapeo $F_\lambda(x) = \lambda x(1-x)$, ¿qué podemos decir de la bifurcación que ocurre en $\lambda_0=1$? ¿Es o no una bifurcación de silla-nodo?

## 4

- Obtengan numéricamente el diagrama de bifurcación del mapeo $Q_c(x)$ a partir de la iteración de una condición inicial en el intervalo de parámetros $-2<c<1/4$. Para esto se necesita iterar un gran número de veces (hasta llegar al ciclo periódico) condiciones iniciales, y repetir esto para *muchos* valores de $c$ en el intervalo. Vale la pena hacer notar que, si hay ciclos periódicos, *a priori* no conocen su periodicidad.

- Para ciertos valores de $c$ característicos, muestren el análisis gráfico del mapeo $Q_c(x)$ a fin de entender los resultados.


## 5

El objetivo de este ejercicio es relacionar, la velocidad de convergencia con que un punto fijo (o una órbita periódica, en el caso de los dos últimos incisos) atraen a puntos suficientemente cercanos, con la derivada del mapeo en el punto fijo (o ciclo periódico). La idea es, entonces, calcular primero el punto fijo y, después, medir cómo la distancia de los iterados sucesivos (de una condición inicial $x_0$) al punto fijo se comporta en el tiempo, para los siguientes mapeos:

- $F(x) = x^2+0.25$

- $F(x) = 3x(1-x)$

- $F(x) = \exp(x-1)$

- $F(x) = x^2 - 1.25$

- $F(x) = \exp(x+1)$

Deberán resolver algunas cosas intermedias. Por ejemplo, ¿qué tanto deben acercarse al punto fijo, a fin de evitar ruido numérico? ¿Qué hay que hacer en el caso en que el punto tenga una estabilidad neutra, i.e., no sea no atractivo ni repulsivo?

En los dos últimos incisos, el interés está en los ciclos de periodo 2.